# Apply ML to Sentiment Analysis

Iremos trabalhar com uma tecnica que faz parte do conjunto __Natural language Processing (NLP)__ chamado _Sentiment Analysis_ , além de entender como usar algoritmos de ML para classificar documentos baseados na sua popularidade. Para isso utilizaremos um dataset do __Intert Movie Database (IMDB)__ 

# Cleaning and Preparing Text Data

O Dataset consiste dos reviews de 50.000 filmes classificados em negativos e positivos, onde positivos significa que o review recebeu mais que 6 estrelas no IMDB e um review negativo se recebeu menos que 5 estrelas.<br>
Os arquivos deste de data consistem de dois diretórios um para o conjunto de treino e outro para o conjunto de teste e em cada um deles mais dois diretórios, um para reviews positivos e outro para negativos. Todos os arquivos estão em txt, iremos então ler estes arquivos e add eles a uma objeto DataFrame do pandas. <br>
Para essa tarefa pode levar algum tempo na maioria dos computadores, devido a natureza da operação e a quantidade de iterações feitas. Sendo assim utilizaremos a lib __PyPrind - Python Progress Indicator__ para acompanhar o progresso dessa operação 

In [1]:
#Libs
import pyprind
import pandas as pd
import os

In [2]:
#Path dos arquivos do dataset
basepath = './aclImdb_v1/'

#Labels dos diretórios
labels = {'pos':1, 'neg':0}

#Barra de progresso
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            
            pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:11:14


O dataset criado possui as labels em ordem dos diretórios acessados, afim de facilitar e tornar mais coerente o split dos dados em Train/Test iremos embaralhar o dataset fazendo uso da função _permutation_ da _np.random_ . Além disso por conveniência iremos converter o dataframe para um arquivo csv

In [3]:
#corrigindo os nomes das colunas
df.columns = ['review', 'sentiment']

import numpy as np

#Cria um gerador com seed(semente inicializadora) zero, ou seja, a cada chamada os mesmos "numero aleatórios" 
#aparecerão.
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [4]:
df.head()

review  sentiment
11841  I was "turned on" to this movie by my flight i...          1
19602  Dresden had great expectations because of its ...          0
45519  Some people seem to think this was the worst m...          0
25747  This movie has a lot of comedy, not dark and G...          1
42642  Jack Black is an annoying character.This is an...          0

# Bag-of-Words

Bag-of-words é uma forma de representar texto em um vetor numerico, basicamente podemos dizer que:
* Criaremos um vocabulario de unico de tokens (por exemplo palavras), de todo o cunjunto de dados
* Construiremos um vetor para cada documento contendo a quantidade de vezes de ocorrência de cada palavra 
<br>

Como as palavras unicas (tokens) em cada documento representam apenas um pequeno subset do conjunto bag-of-words, os vetores consistiram principalmente de zeros, e por isso são chamados de vetores (ou matriz) esparsos.

## Transformando palavras em um vetor de features

Para a construção de tal vetor podemos utilizar a classe _CountVectorizer_ da lib scikit-learn, tal classe toma uma array de texto, podendo ser documentos ou frases, e construi um modelo de bag-of-words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
count = CountVectorizer()

docs = np.array(['The sun is shining', 
                 'The weather is sweet', 
                 'The sun in shining and the weather is sweet'])
bag = count.fit_transform(docs)

Ao chamar o metódo _fit_transform_ nós construimos um vocabulario que será nossa bag-of-words e transformamos as frases passadas em vetores esparsos

In [7]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 2, 'shining': 3, 'weather': 7, 'sweet': 5, 'in': 1, 'and': 0}


Como podemos ver o vocabulario é um dicionario que mapeia as palavras unicas a um indice inteiro. <br><br>

Veremos agora o vetor criado:

In [8]:
print(bag.toarray())

[[0 0 1 1 1 0 1 0]
 [0 0 1 0 0 1 1 1]
 [1 1 1 1 1 1 2 1]]


Cada posição (indice) no vetor de features, corresponde ao valor inteiro armazenado no dicionario de itens _CountVectorizer_ , por exemplo a primeira feature do vetor cujo indice é 0 assemelha-se a palavra 'and' que ocorre apenas no ultimo documento. Os valores do vetor de features são chamados de __raw term frequencies:__ $tf(t,d)$ , o numero de vezes que um ter $t$ ocorre em um documento $d$

___Note:___ A sequencia de itens da bag-of-words criada é chamada de modelo __1-gram__ ou __unigram__ onde cada item ou token do vocabulario corresponde a uma unica palavra. De maneira mais geral, uma sequencia de itens em __NLP__ - palavras, letras ou simbolos - são chamados de __n-grams__ . A escolha do numero n do modelo n-gram depende da aplicação. Como exemplo, podemos dizer então que:
* __1-gram:__ "the", "sun", "is", "shining"
* __2-gram:__ "the sun", "sun is", "is shining"

A classe _CountVectorizer_ do scikit-learn permite utilizar facilmente modelos n-grams através do parametro _ngram_range_ , onde 1-gram é usado por padrão.

## Avaliação da relevância das palavras por meio da frequência do termo inversa a frequência por documento

No contexto deste projeto, assim como em muitos outros, encontraremos palavras que aparecem em varios documentos sendo das duas classes presentes (positiva e negativa) e este tipo de ocorrência não contem muitas informações descriminatórias uteis. Iremos então estudar uma tecnica útil  chamada __"Term frequency-inverse document frequency" (tf-idf)__ ( frequência do termo–inverso da frequência nos documentos) , que pode ser usada para diminuir o peso dessas palavras que ocorrem frequentemente no vetor de features, ou seja, diminui o peso da palavra de um documento com relação a todos os outros documentos. <br>
Tf-idf pode ser definido como: 
\begin{equation*}
    tfidf(t,d) = tf(t,d) * idf(t,d)
\end{equation*}


Onde $tf(t,d)$ é a frequência de um termo em um documento visto anteriormente e $idf(t,d)$ é a frequencia de documento inversa, e é calculada como:

\begin{equation*}
    idf(t, d) = log[\frac{n_d}{1+df(d,t)}]
\end{equation*}

Onde, $n_d$ é o numero total de documentos e $df(d,t)$ é o numero de documentos $d$ que contem o termo $t$

O termo $1$ no demoninador serve para não haver divisão por zero caso a função $df$ retorne zero. O $log$ é usado para garantir que baixas frequencia de documentos não sejam dado muito peso.<br>
<br>
Esta transformação esta implementada no _scikit-learn_ como uma classe, __TfdifTransformer__ , que recebe os _raw terms frequencies_ do _CountVectorizer_ e então transforma-os com tf-idf 

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

"""
norm: Normalização para cada linha do output. l2 siginica Soma dos quadrados = 1, ou seja, o angulo entre
      dois vetores é 1.

use_idf: Determina a utilização do Idf

smooth_idf: Previni divisões por zero add 1 aos pesos das frequencias dos documentos
"""

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.   0.43 0.56 0.56 0.   0.43 0.  ]
 [0.   0.   0.43 0.   0.   0.56 0.43 0.56]
 [0.41 0.41 0.24 0.31 0.31 0.31 0.48 0.31]]


__TfidTransformer__ após fazer o produtor vetorial entre $df$ e $idf$, através do parametro _norm_ normaliza o resultado, isso garante uma uniformidade nos valores além de impedir uma possível divisão por zero. Neste caso utilizamos a normalização por __L-2 normalization__ (l2) que consite da divisão do vetor candidato a normalização pela sua norma L2:

\begin{equation*}
    V_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_1^2 + v_2^2 + ... + v_n^2}} = \frac{v}{\sqrt{\sum_{i=1}^n v_i^2}}
\end{equation*}


# Limpeza dos dados de texto

O próximo passo após construir nossa bag-of-words consiste em limpar nossos dados para remover os caracteres indesejados. Para ilustrar a importancia disso, vejamos um exemplo de como estão os textos de um dos nossos documentos:

In [10]:
df.loc[0, 'review'][-50:]

" play it, enjoy it, love it. It's PURE BRILLIANCE."

Há muitos simbolos de pontuação, assim como podem haver marcações de HTML em outros documentos. Sendo assim retirarei todos esses pontos e marcações, com excessão de possiveis emoticons como ":)" que podem ser uteis para a analise de sentimetno. <br>
Para esta tarefa utilizaremos a biblioteca de expressões regulares do Python __Python Regular Expression (regex)__

In [11]:
#Regex
import re

In [12]:
#Função para remover as pontuações e marcadores de html
def preprocessor(text):
    
    #regex para remover marcadores html
    text = re.sub('<[^>]*>', '', text)
    
    #armazenando os caracteres de emoticons para usa-los futuramente
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    #removendo todas as "não-palavras" e convertendo todo o texto para minusculo
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    
    return text

__Note:__ Não estamos levando em conta a _capitalização_ das palavras, ou seja, a ordem em que elas aparencem na frase, assumindo assim que esta informação não tem relevância para esta analise.

__Note:__ Para saber mais sobre expressões regulares:
* Tutorial (muito bom) da Goggle: https://developers.google.com/edu/python/regular-expressions
* Documentação Python: https://docs.python.org/3.6/library/re.html

In [13]:
#Verificando de a função preprocessor funciona adequadamente
preprocessor(df.loc[0, 'review'][-50:])

preprocessor("</a>This :) is :(a test :-)!")

'this is a test :) :( :)'

Como esta funcionando bem, vamos então aplicar a função a todos os nossos dados:

In [14]:
df['review'] = df['review'].apply(preprocessor)

# Processando Documentos em Tokens

Uma maneira de empregar essa tecnica de _tekenize_ documentos, consiste em dividir o documentos em palavras individuais, separando-as pelos espaços em branco.

In [15]:
#Função tokenize
def tokenizer(text):
    return text.split()

#Testando
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

Uma outra tecnica possível é a chamada __Word Stemming__ , que consiste em transformar uma palavra em sua "forma raiz", ou seja, reconhecendo que aquela é uma palavra derivada e através do seu sufixo encontrando a sua forma raiz. Esta tecnica foi criada por _Martin F. Porter_ que a chamou de __Porter Stemmer Algorithm__ em 1979. Anos mais tarde foi criada a [__Natural Language Toolkit (NLTK)__](http://www.nltk.org/) que implementa este algoritmo.

In [16]:
#Porter Stemmer from NLTK
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

#Testando
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

Como vemos o novo tokenizador retorna palavras derivadas ("running") para sua forma raiz ("run")

__Note:__ Porter Stemmer ja é considerado bem antigo e há outros algoritmos mais modernos para fazer a mesma tarefa de forma mais eficiente como o __Snowball Stemmer__ e o __Lancaster Stemmer__, ambos também implementados na lin _NLTK_ .<br>
Existe uma outra tecnica bem mais agressiva que a de _Stemming_ e que também minimiza os erros gerados, como de palavras que não existem, chamado de __Lemmatization__ que retorna as palvras para sua forma caninca, ou seja, sua forma raiz mas correta gramaticalmente, tal tecnica é bem mais dificil de implementar e alguns estudos apontam pouco impacto na performance em modelos de classificação de texto - ver: (Influence of Word Normalization on Text Classification, Michal Toman, Roman Tesar, and Karel Jezek, Proceedings of InSciT, pages 354–358, 2006).

## Stop-Words

Stop-words são palavras muito comuns em textos que não necessariamente contribuem para uma classificação do texto, são palavras de conexão e de ligação entre as frases, por exemplo do inglês: _is, and, has, like_ <br>
A remoção dessas stop-words é útil quando estamos trabalhando com a raiz das palavras ou termos que tenham sua frequencia normalizada ao invés de Tfidf em que ja é diminuido a frequencia de ocorrência das palavras.

Para esta tarefa iremos utilizar um conjunto de dados contendo 127 stop-words (do Inglês) disponíveis na propria lib _NLTK_

In [17]:
import nltk

#Fazendo o download da própria lib do conjunto de stop-words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
#Aplicando as stop-words baixadas
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a  lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

__As stop-words foram removidas e o texto foi transformado para sua forma raiz e dividido em palavras unicas__

# Treinando um modelo de Regressão Logistica para Classificação de Documentos

Primeiramento vamos dividir nosso conjunto de dados (já limpo) em conjunto de treino e de test; dividiremos em 50% para cada conjunto

In [19]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [20]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(20448,) (20448,) (29553,) (29553,)


Utilizaremos agora o __GridSearchCV__ para fazer uma busca pelos melhores parametros para nosso modelo de regressão logistica, usando o metodo de validação cruzada com 5-folds estratificados. <br>
Além disso utilizaremos a biblioteca pipeline que facilita todo o trabalho de transformações e mudanças de parametros nos modelos

In [21]:
#libs
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
#Normalizando a frequencia das palavras no documento
#Parametros com None por que os documentos ja foram limpos
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

#Parametros a serem testamos
"""
Chaves:
    ngram_range: Range de palavras q serão consideradas unicas (1-gram, 2-gram, etc...), referente a função de 
                 analise de frequencia de termos
    stop_words:  lista com as palavras de ligação e/ou sem valor semantico
    tokenizer:   Função que fara o split do texto e transformará as palavras para sua raiz
    penalty:     Método de normalização dos resultados do modelo
    C:           Inverso da "força" de regularização, valores menores indicam uma regularização maior dos dados
    use_idf:     Habilita o rebalanceamento inverso da frequencia de termo por documento
    norm:        Normalização do produto vetorial (usado na analise de frequencia de termos)
"""
param_grid = [{'vect__ngram_range':[(1,1)], 
               'vect__stop_words':[stop, None], 
               'vect__tokenizer':[tokenizer, tokenizer_porter], 
               'clf__penalty':['l1', 'l2'], 
               'clf__C':[1.0, 10.0, 100.0]}, 
             
              {'vect__ngram_range':[(1,1)], 
              'vect__stop_words':[stop, None], 
              'vect__tokenizer':[tokenizer, tokenizer_porter], 
              'vect__use_idf':[False], 
              'vect__norm':[None], 
              'clf__penalty':['l1', 'l2'], 
              'clf__C':[1.0, 10.0, 100.0]}
             ]

#Pipeline
"""
Parametros: Passados apartir de um dicionario com chaves e valores que serão testados nos modelos
    Lista de tuplas:
        Primeiro argumento: (Nome, Transformação) - Transformação sobre os dados do modelo
        Segundo argumento: (Modelo) - Modelo a ser implementado
"""
lr_tfidf = Pipeline([('vect', tfidf), 
                     ('clf', LogisticRegression(random_state=0))])

#GridSearchCV
"""
Parametros (respectivamente):
    estimator (lr_tfidf): interface que relacionará o modelo aos parametros que serão testados, neste caso
                          a função pipeline, mas poderiam ser outros modelos com parametros diferentes
                          
    param_grid:           Dicionarios com o nomes dos parametros como chaves (string) e uma lista como valores
    
    scoring:              Metrica para avaliação do modelo
    
    cv:                   Um inteiro como gerador das divisões para uma validação cruzada
    
    verbose:              Quanto maior o valor mais mensagens exibidas durante a busca
    
    n_jobs:               Números de tarefas que serão executadas em paralelo, afeta os nucleos do processador 
                          que serão usados. Utiliza '-1' fará uso de todos os nucleos, acelerando o processo
    
"""
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [26]:
#Aplicando o gridsearchcv nos dados de treino
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 86.7min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 110.8min finished
/home/leandro/.conda/envs/Data_Science/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                         

___A operação acima levou em torno de 45min para ser completada!!!___

Na Busca do GridSearchCV utilizamos o __TfidfVectorizer__ pois ele faz o mesmo que as outras duas classes usadas anteriormente, CountVectorizer e TfidfTransformer. <br>
_A saber:_
* __CountVectorizer:__ Convert a collection of text documents to a matrix of token counts
This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

* __TfidfTransformer:__ Transform a count matrix to a normalized tf or tf-idf representation

* __TfidfVectorizer:__ Convert a collection of raw documents to a matrix of TF-IDF features

O ___param_grid___ utilizado consiste de dois dicionarios como parametros. No primeiro utilizamos as configurações padrões do _ TfidfVectorizer_ (use_idf=True, smooth_idf=True, and norm='l2') para calcular a tf-idf; no segundo dicionario mudamos alguns parametros (use_idf=False, smooth_idf=False, and norm=None).

## Printando os melhores parametros encontrado pelo GridSearchCV

In [30]:
print('Best parameter set: %s' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f2188755440>}


Temos então que, por exemplo, os melhores parametros são a função de tokenização simples (sem a implementação Porter), não se faz necessário as stop-words e usando a tf-idf em combinação com LogisticRegression com L2-regularization, com uma força de regularização de C=10.0

## Evaluation

Utilizando o melhor modelo encontrando pelo grid search, veremos a média da accuracy após uma validação cruzada de 5-folds nos dados de treino e accuracy da classificação no conjunto de test:

In [32]:
#Accuracy nos dados de treino
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy: 0.895


In [34]:
#Treinando o modelo com os melhores parametros
clf = gs_lr_tfidf.best_estimator_

#Accuracy nos dados de test
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.893


O resultado revela que o modelo de Machine Learning consegue prever quando uma avaliação de um filme é positiva ou negativa com __~90% de acurácia__ .

# Working with bigger data – online algorithms and out-of-core learning

Como visto na analise anterior, trabalhar com analise de sentimento de um conjutno de dados tão grande (50.000 reviews) pode ser custoso computacionalmente. Em muitas aplicações do mundo real não é incomum lidar com banco de dados maiores ainda e que as operações sobre eles podem exceder facilmente a capacidade de memória dos cumputadores. Para resolver este problema estudaremos uma tecnica chamada de Out-Of-Core Learning , que nos permite trabalhar com banco de dados grandes através de pequenos fits do modelo classificador em "parcelas" menores do banco de dados, de forma incremental.

Para tal tarefa utilizaremos a função partial_fit do SGDClassifier do scikit-learn, para "transmitir" os documentos diretamente do local drive, e treinar um modelo de Logistic Regression usando pequenos pacotes de documentos

Primeiro de tudo iremos definir uma função _tokenizer_ para limpar os documentos não processados do __movie_data.cvs__

In [22]:
#Libs
import numpy as np
import re
from nltk.corpus import stopwords

In [23]:
stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Agora definiremos uma função __stream_docs__ para ler e retornar um documento por vez

In [24]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) #skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [25]:
#Testando a função
next(stream_docs(path='movie_data.csv'))

('"I was ""turned on"" to this movie by my flight instructor and now I wonder how the heck it was out there for nearly five years before I finally discovered it. If you have any love of flying at all, especially an attachment to the planes of WWII, this is an absolute must see, vastly superior to the pathetic ""Pearl Harbor"" and up there in rivalry with the famed ""Battle of Britain"" filmed more than thirty years ago. There are moments when you feel as if you are flying wingman, literally dodging the shell casings of your leader as you roll in on a Me 109 or He 111. <br /><br />As an historian this film deeply touched me as well for it is about the plight endured by tens of thousands of gallant Poles, Hungarians, Slovaks and Czechs who in 1939-1940 fled their homelands, made it to England, fought with utmost bravery for the survival of western civilization, and then were so callously abandoned by ""us"" after the war when they were arrested by the communists upon their return to thei

Iremos agora definir uma função para retornar apenas uma porção do documento selecionado, determinado por um parametro _size_ :

In [26]:
#A função next() itera sobre uma lista, retornando o proximo elemento, a cada chamada ele pula 
#o elemento anteriormente retornado e vai para o proximo

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

Não podemos usar função _CountVectorizer_ por ela precida criaf todo um vocabulario na memória, e nem mesmo a _TfidfVectorizer_ por que ela precisa manter na memoria todos os vetores de vocabularios para operar sobre eles, e nossas função criadas aqui operam apenas uma vez sobre cada linha do documento. Entretanto há no scikit-learn uma outra classe para fazer a vetorização para processamento de texto chamada __HashingVectorizer.HashingVectorizer__ , trata dados independentes e utiliza tecnicnas de _hashing_

In [27]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None, tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter_no_change=1)
doc_stream = stream_docs(path='movie_data.csv')

Inicializamos então o HashingVectorizer com um numero grande de features, isso fará com que diminua a chance de " _colisões de hash_ ", porem aumenta o numero de coeficientes do modelo de regressão logistica.

__Vamos agora, de fato inicia o método de Out-Of-Core Learning:__

In [28]:
import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0,1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


Iniciamos nossa barra de medição(pbar) com 45 interações e no _loop_ iteramos sobre 45 mini-baches (porções pequenas dos documentos), onde cada mini-bache consiste de 1000 documentos, ou seja teremos então 45000 documentos processados ao final. Os outros 5000 usaremos para avaliar o mdoelo treinado com _partial_fit_

In [29]:
#Evaluation

#Add os 5k documentos restantes aos conjuntos de testes
X_test, y_test = get_minibatch(doc_stream, size=5000)

#Vetorizando os documentos co HashingVectorizer
X_test = vect.transform(X_test)

#Print do score
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


Podemos ver que tivemos uma acuracia ligeiramente menor __(~88%)__ do que o modelo treinado com GridSearchCV, porem o ganho computacional foi enorme, em menos de 1min o código foi executado, provando assim que o método de Out-of-Core Learning é bem eficiente e entrega resultados equivalentes aos outros.
<br><br>
Podemos por fim atualizar nosso modelo, treinando-o com os 5k documentos restantes:

In [30]:
#Finalizado o Treinamento incremental do modelo
clf = clf.partial_fit(X_test, y_test)

__Note:__ Hoje em dia a um algoritmo mais moderno para criar uma _bag-of-words_ , é um modelo chamado de [__word2vec__](https://code.google.com/p/word2vec/) , um algoritmo que o Google disponibilizou em 2013. Word2vec é um modelo de aprendizagem não-supervisionada baseado em redes neurais com aprendizado automático sobre o relacionamento das palavras. A idéia por trás deste algoritmo é colocar as palavras com significados similares dentro de _clusters_ similares, e através de um "espaço vetorial inteligente", o modelo reproduz certas palavras usando uma simples matematica de vetores, por exemplo: _king - man + woman = queen_


# Serializando o modelo out-of-core, para a próxima seção que envolve o deploy

__Pickle__ serve para serializar o modelo, ou seja, "salvar" o estado atual do modelo para futuramente poder usa-lo sem a necessidade de retreina-lo

In [31]:
import pickle
import os

In [32]:
"""
Criação dos diretórios:
            movieclassifier: armazenará os arquivos e dados da aplicação web
            pkl_objects: Um subdiretório para salvar os objetos python serializados
"""
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
    
"""
O método Dump irá serializar o modelo de logistic regression treinado anteriormente, assim como as stop-words
usadas da lib NLTK, desta forma não será preciso instalar a NLTK no servidor.

O primeiro argumento refere-se ao objeto que será serializado.
O segundo argumento é um arquivo aberto onde o objeto Pyhton será escrito
    Através do argumento 'wb' na função open, o arquivo é aberto em modo binario pelo pickle
    O argumento protocol=4 escolhe o ultimo e mais eficiente protocolo implementado pelo pickle apartir da versão
        do Python 3.4
"""
pickle.dump(stop, 
                open(os.path.join(dest, 'stopwords.pkl'), 'wb'), 
                protocol=4)

pickle.dump(clf, 
               open(os.path.join(dest, 'classifier.pkl'), 'wb'), 
               protocol=4)

__Diretório movido para a Pasta do chapter 9__

O modelo de Regressão Logistica treinado possui diversos numpy arrays e uma maneira mais eficiente de serializar numpy arrays é com a biblioteca __joblib__ , porem por questão de compatibilidade com o servidor que iremos utilizar para hospedar a aplicação utilizaremos o __pickle__

# Modelagem de Tópicos com "Latent Dirichlet Allocation"

Modelagem de tópicos descreve uma ampla gama de tarefas sobre documentos de texto sem labels. Um tipico exemplo é a categorização de documentos dentro de um grande conjunto de artigos de jornais, onde não sabemos a quais categorias os artigos pertencem. Em modelgame de tópicos buscamos determinar a qual categoria os artigos pertencem - por exemplo, esportes, finanças, news, politica, etc... No contexto de problemas de ML envolvendo categorização podemos considerar a _Modelagem de Tópicos_ como um problema de _Clusterização_ , uma subcategoria do Aprendizado Não-Supervisionado.

Iremos estudar a técnica de modelagem de tópico chamada __Latent Dirichlet Allocation__ comumente abreviada para __LDA__ , porem não devemos confundir com uma outra técnica: __Linear Discriminant Analysis__ que trata-se de uma aprendizagem supervisionada de redução de dimensionalidade.

## LDA com Scikit-Learn

Iremos utilizar o LDA do scikit-learn para decompor nosso dataset de review de filmes e categoriza-lo em diferentes topicos. Restringimos, inicialmente, a analise para 10 tópicos, porem é possível experimentar outros hiperparametros do algoritmo para explorar outros topicos.<br>
Então vamos começar do começo:

In [55]:
#Load dataset
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head()

review  sentiment
0  I was "turned on" to this movie by my flight i...          1
1  Dresden had great expectations because of its ...          0
2  Some people seem to think this was the worst m...          0
3  This movie has a lot of comedy, not dark and G...          1
4  Jack Black is an annoying character.This is an...          0

Criaremos agora uma matrix de _bag-of-word_ com o __CountVectorizer__ para dar como entrada no __LDA__ . Por conveniencia iremos utilizar o stopwords

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

"""
Parametros:
    max_df: Cria um vocabulario ignorando os termos com uma frequencia maior que o limite determinado
    
    max_features: Se não for None, cria um vocabulario que considera apenas os principais max_features, ordenados
                  por termos de frequencia nos documentos
"""
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)

X = count.fit_transform(df['review'].values)

Definimos a frequência maxima de palavras do documento a ser considerada como 10% (max_df=.1), para excluir palavras que ocorrem com muita frequencia nos documentos. A lógica por tras dessa remoção é que palavras que ocorrem com uma frequencia alta em todos os documentos, possivelmente não contribuem para uma determinação da categoria do documento, por serem palavras genericas. Também reduzimos o numero de palavras para ser consideradas para as 5000 mais frequentes (max_features=5000), para limitar a dimensionalidade do dataset e melhorar a perfomance do LDA. Entretando tanto o _max_def_ quanto _max_features_ são hiperparametros escolhidos arbitrariamente e ambos podem ser tunados.

In [62]:
#Implementando o LDA
from sklearn.decomposition import LatentDirichletAllocation

#Instanciando o modelo
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')

#Fit
X_topics = lda.fit_transform(X)

O método __LDA__ configurado da maneira acima,  leva em torno de 5min para rodar em pcs normais 

Definindo o _learning_method = 'batch'_ deixamos o lda fazer sua estimativa baseado em todo o conjunto de treino disponível (matrix de bag-of-words) em uma iteração, o que torna a execução mais lenta do que seria com o método _'online'_ porem desta forma teremos um resultado mais acurado. <br>
<br>
__OBS:__ A escolha do metodo de aprendizagem 'online' é semelhante ao 'mini-batch' criado anteriormente.

Podemos verificar agora a matrix gerada pelo LDA contendo a importancia das palavras (neste caso 5000) para cada um dos 10 topicos em ordem crescente: 

In [63]:
lda.components_.shape

(10, 5000)

In [64]:
#Listando as 5 palavras mais importantes para distinguir cada topico
#Lembrando que as palavras são retornadas em ordem crescente, então para exibir um Top 5 devemos reverter o vetor

n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1 :-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father girl women
Topic 3:
american dvd music tv early
Topic 4:
audience human cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex gore blood
Topic 7:
role performance comedy actor performances
Topic 8:
series war episode episodes season
Topic 9:
book version original effects read
Topic 10:
action guy fight guys music


Baseado nas palavras mais importantes de cada tópico, podemos assumir que o LDA identificou os seguinte topicos:
<ol>
    <li>Filmes ruins em geral</li>
    <li>Filmes sobre familias</li>
    <li>Shows de TV</li>
    <li>Filmes sobre Arte</li>
    <li>Filmes policiais</li>
    <li>Filmes de Terror</li>
    <li>Filmes de Comédia</li>
    <li>Séries de guerra (talvez)</li>
    <li>Filmes baseados em livros</li>
    <li>Filmes de ação</li>
</ol>

Podemos testar se esses tópicos identificados pelo modelo fazem algum sentido exibindo, por exemplo, 3 reviews de filmes que seriam da categoria _Terror_ (Topic=6, mas com indice igual a 5 por ser um vetor) e verificar se os reviews condizem com filmes de terror:

In [67]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorro Movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horro Movie #1:
Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings tha ...

Horro Movie #2:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horro Movie #3:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...


Aparentemente os reviews condizem com a categoria!!! Very nice!!!!!!!!!!!!